In [ ]:
#@title Installing packages
!pip install apyori

In [2]:
#@title Preparing the data
#Importing libraries
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('Data.csv', header = None) # No columns

# Preparing the data for apyori as list
shopping_list = []
for i in range(df.shape[0]):
  shopping_list.append([str(product) for product in df.iloc[i, :]])

In [3]:
#@title Training the Apriori model
from apyori import apriori
"""
min_support = 3*7 / 7501 # A product is sold at least 3 times per day
min_confidence = 0.2, starts from 0.8 --> no rule, 0.4 --> a few, 0.2 --> good!
min_lift = 3 from experience for relevant rules
min_length and max_length for the amount of products = 2 for "Buy 1 Get 1 Free"   
"""
# apriori model returns a list of rules
rules = apriori(transactions = shopping_list, min_support = 0.003, 
                min_confidence = 0.2, 
                min_lift = 3,
                min_length = 2,
                max_length = 2)

In [4]:
#@title Results
def inspect(rules):
  """
  Takes rules as a list of return object of apriori model (generator),
  Returns a pandas DataFrame object for decent illustration and 
  sorted final rules by descending lift values.  
  """
  # Separate the columns
  lhs         = [tuple(r[2][0][0])[0] for r in rules]
  rhs         = [tuple(r[2][0][1])[0] for r in rules]
  supports    = [r[1] for r in rules]
  confidences = [r[2][0][2] for r in rules]
  lifts       = [r[2][0][3] for r in rules]
  # Collect the columns in one result
  result = list(zip(lhs, rhs, supports, confidences, lifts))
  # Convert to pandas DataFrame
  result = pd.DataFrame(result, columns = ['Left Hand Side', 
                                           'Right Hand Side', 
                                           'Support', 
                                           'Confidence', 
                                           'Lift'])
  return result.nlargest(n=10, columns='Lift')

final_rules = inspect(list(rules))
final_rules.style


,Left Hand Side,Right Hand Side,Support,Confidence,Lift
3,fromage blanc,honey,0.003333,0.245098,5.164271
0,light cream,chicken,0.004533,0.290598,4.843951
2,pasta,escalope,0.005866,0.372881,4.700812
8,pasta,shrimp,0.005066,0.322034,4.506672
7,whole wheat pasta,olive oil,0.007999,0.271493,4.122410
5,tomato sauce,ground beef,0.005333,0.377358,3.840659
1,mushroom cream sauce,escalope,0.005733,0.300699,3.790833
4,herb & pepper,ground beef,0.015998,0.323450,3.291994
6,light cream,olive oil,0.003200,0.205128,3.114710
